In [3]:
model_path ="mistralai/Mistral-7B-Instruct-v0.2"

In [4]:
model_kwargs = {
        "load_in_4bit" : True           
}

In [5]:
from transformers import AutoModel, AutoConfig, BitsAndBytesConfig

In [6]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [18]:
for nm in model.named_modules(): print(nm[0])


embed_tokens
layers
layers.0
layers.0.self_attn
layers.0.self_attn.q_proj
layers.0.self_attn.k_proj
layers.0.self_attn.v_proj
layers.0.self_attn.o_proj
layers.0.self_attn.rotary_emb
layers.0.mlp
layers.0.mlp.gate_proj
layers.0.mlp.up_proj
layers.0.mlp.down_proj
layers.0.mlp.act_fn
layers.0.input_layernorm
layers.0.post_attention_layernorm
layers.1
layers.1.self_attn
layers.1.self_attn.q_proj
layers.1.self_attn.k_proj
layers.1.self_attn.v_proj
layers.1.self_attn.o_proj
layers.1.self_attn.rotary_emb
layers.1.mlp
layers.1.mlp.gate_proj
layers.1.mlp.up_proj
layers.1.mlp.down_proj
layers.1.mlp.act_fn
layers.1.input_layernorm
layers.1.post_attention_layernorm
layers.2
layers.2.self_attn
layers.2.self_attn.q_proj
layers.2.self_attn.k_proj
layers.2.self_attn.v_proj
layers.2.self_attn.o_proj
layers.2.self_attn.rotary_emb
layers.2.mlp
layers.2.mlp.gate_proj
layers.2.mlp.up_proj
layers.2.mlp.down_proj
layers.2.mlp.act_fn
layers.2.input_layernorm
layers.2.post_attention_layernorm
layers.3
layers.

In [7]:
model = AutoModel.from_pretrained(model_path, load_in_4bit=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
print_trainable_parameters(model)

trainable params: 131338240 || all params: 3620999168 || trainable%: 3.63


In [13]:
from peft import LoraConfig, get_peft_model

In [19]:
lora_config = LoraConfig(
    r = 16, 
    lora_alpha= 16, 
    target_modules= ["q_proj", "v_proj"],
    lora_dropout=.1, 
    bias="none"
)
    

In [20]:
lora__model = get_peft_model(model, lora_config)

In [21]:
from transformers import TrainingArguments, Trainer


In [22]:
batch_size = 1;

In [24]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "perfModel", 
    evaluation_strategy="epoch", 
    save_strategy="epoch", 
    learning_rate=5e-3, 
    num_train_epochs=5, 
    per_device_eval_batch_size=batch_size, 
    gradient_accumulation_steps=2, 
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)


In [ ]:
trainer = Trainer(
    lora_model, 
    args, 
    train_dataset=
    